# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 27 2023 8:30AM,258154,10,0086431418,ISDIN Corporation,Released
1,Feb 27 2023 8:30AM,258154,10,0086431983,ISDIN Corporation,Released
2,Feb 27 2023 8:30AM,258154,10,0086431984,ISDIN Corporation,Released
3,Feb 27 2023 8:30AM,258154,10,0086432563,ISDIN Corporation,Released
4,Feb 27 2023 8:30AM,258154,10,0086432564,ISDIN Corporation,Released
5,Feb 27 2023 8:30AM,258157,10,9201413,ISDIN Corporation,Released
6,Feb 27 2023 8:30AM,258154,10,0086430667,ISDIN Corporation,Released
7,Feb 27 2023 8:30AM,258154,10,0086430670,ISDIN Corporation,Released
8,Feb 27 2023 8:30AM,258154,10,0086430685,ISDIN Corporation,Released
9,Feb 27 2023 8:30AM,258154,10,0086430698,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
11,258149,Released,18
12,258153,Executing,1
13,258153,Released,30
14,258154,Released,44
15,258157,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258147,NaN,1.0,NaN
258149,NaN,NaN,18.0
258153,NaN,1.0,30.0
258154,NaN,NaN,44.0
258157,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258089,0.0,0.0,5.0
258099,16.0,7.0,0.0
258115,0.0,9.0,2.0
258122,7.0,0.0,18.0
258131,0.0,0.0,13.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258089,0,0,5
258099,16,7,0
258115,0,9,2
258122,7,0,18
258131,0,0,13


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258089,0,0,5
1,258099,16,7,0
2,258115,0,9,2
3,258122,7,0,18
4,258131,0,0,13


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258089,,,5
1,258099,16,7,
2,258115,,9,2
3,258122,7,,18
4,258131,,,13


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 27 2023 8:30AM,258154,10,ISDIN Corporation
5,Feb 27 2023 8:30AM,258157,10,ISDIN Corporation
42,Feb 27 2023 8:30AM,258153,10,ISDIN Corporation
72,Feb 27 2023 8:30AM,258149,10,ISDIN Corporation
94,Feb 24 2023 2:52PM,258147,19,ACG North America LLC
95,Feb 24 2023 2:42PM,258146,10,Bio-PRF
98,Feb 24 2023 2:37PM,258145,10,"Digital Brands, LLC"
103,Feb 24 2023 11:34AM,258131,10,ISDIN Corporation
116,Feb 24 2023 10:41AM,258122,10,ISDIN Corporation
141,Feb 24 2023 9:13AM,258115,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 27 2023 8:30AM,258154,10,ISDIN Corporation,,,44
1,Feb 27 2023 8:30AM,258157,10,ISDIN Corporation,,,1
2,Feb 27 2023 8:30AM,258153,10,ISDIN Corporation,,1,30
3,Feb 27 2023 8:30AM,258149,10,ISDIN Corporation,,,18
4,Feb 24 2023 2:52PM,258147,19,ACG North America LLC,,1,
5,Feb 24 2023 2:42PM,258146,10,Bio-PRF,,,3
6,Feb 24 2023 2:37PM,258145,10,"Digital Brands, LLC",,5,
7,Feb 24 2023 11:34AM,258131,10,ISDIN Corporation,,,13
8,Feb 24 2023 10:41AM,258122,10,ISDIN Corporation,7,,18
9,Feb 24 2023 9:13AM,258115,10,Bio-PRF,,9,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 27 2023 8:30AM,258154,10,ISDIN Corporation,44,,
1,Feb 27 2023 8:30AM,258157,10,ISDIN Corporation,1,,
2,Feb 27 2023 8:30AM,258153,10,ISDIN Corporation,30,1,
3,Feb 27 2023 8:30AM,258149,10,ISDIN Corporation,18,,
4,Feb 24 2023 2:52PM,258147,19,ACG North America LLC,,1,
5,Feb 24 2023 2:42PM,258146,10,Bio-PRF,3,,
6,Feb 24 2023 2:37PM,258145,10,"Digital Brands, LLC",,5,
7,Feb 24 2023 11:34AM,258131,10,ISDIN Corporation,13,,
8,Feb 24 2023 10:41AM,258122,10,ISDIN Corporation,18,,7
9,Feb 24 2023 9:13AM,258115,10,Bio-PRF,2,9,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 27 2023 8:30AM,258154,10,ISDIN Corporation,44,,
1,Feb 27 2023 8:30AM,258157,10,ISDIN Corporation,1,,
2,Feb 27 2023 8:30AM,258153,10,ISDIN Corporation,30,1,
3,Feb 27 2023 8:30AM,258149,10,ISDIN Corporation,18,,
4,Feb 24 2023 2:52PM,258147,19,ACG North America LLC,,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 27 2023 8:30AM,258154,10,ISDIN Corporation,44.0,NaN,NaN
1,Feb 27 2023 8:30AM,258157,10,ISDIN Corporation,1.0,NaN,NaN
2,Feb 27 2023 8:30AM,258153,10,ISDIN Corporation,30.0,1.0,NaN
3,Feb 27 2023 8:30AM,258149,10,ISDIN Corporation,18.0,NaN,NaN
4,Feb 24 2023 2:52PM,258147,19,ACG North America LLC,NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 27 2023 8:30AM,258154,10,ISDIN Corporation,44.0,0.0,0.0
1,Feb 27 2023 8:30AM,258157,10,ISDIN Corporation,1.0,0.0,0.0
2,Feb 27 2023 8:30AM,258153,10,ISDIN Corporation,30.0,1.0,0.0
3,Feb 27 2023 8:30AM,258149,10,ISDIN Corporation,18.0,0.0,0.0
4,Feb 24 2023 2:52PM,258147,19,ACG North America LLC,0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2581361,134.0,15.0,7.0
19,516246,0.0,8.0,16.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2581361,134.0,15.0,7.0
1,19,516246,0.0,8.0,16.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,134.0,15.0,7.0
1,19,0.0,8.0,16.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,134.0
1,19,Released,0.0
2,10,Executing,15.0
3,19,Executing,8.0
4,10,Completed,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19
Status,,
Completed,7.0,16.0
Executing,15.0,8.0
Released,134.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19
0,Completed,7.0,16.0
1,Executing,15.0,8.0
2,Released,134.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19
0,Completed,7.0,16.0
1,Executing,15.0,8.0
2,Released,134.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()